## Imports and dataset

In [ ]:
from src.models.conjugate_bayes_lin_reg import NormalInverseGammaPriorLinearRegression
from src.models.bayesian_nn import BayesianNN
from src.utils import id, expy2, l1_projection, plot_ppds

from src.attacks.point_attacks import attack, attack_fgsm

import numpy as np
import torch
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid", palette="muted", font="serif")

sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.rcParams.update({
    'axes.titlesize': 18,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'axes.titleweight': 'bold',
    'axes.edgecolor': 'black',
    'axes.linewidth': 1,
    'grid.alpha': 0.5,
    'grid.linestyle': '--',
    'legend.fontsize': 12,
    'legend.frameon': False,
    'figure.dpi': 300,  
})

In [ ]:
# set all seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# Wine dataset

In [4]:
# Wine Quality dataset with response in {3, 4, 5, 6, 7, 8}, 11 features and 4898 samples

# URL to the Wine Quality dataset (for example, from UCI Machine Learning Repository)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
# Load the dataset directly into a Pandas DataFrame
data = pd.read_csv(url, delimiter=";")

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [5]:
# Normalize the data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=42)

In [6]:
import numpyro
numpyro.set_host_device_count(88)

In [ ]:
# Fit the model
model = BayesianNN(input_dim=X_train.shape[1], hidden_units=3)
try:
    model.load('../src/models/bayesian_3nn')
except FileNotFoundError:   
    print('Not model found, fitting the model') 
    model.fit(
        X_train, 
        y_train,
        num_warmup=500,
        num_samples=1000,
        num_chains=88,
    )
    model.save('../src/models/bayesian_3nn')

In [ ]:
model.sample_predictive_distribution(
    torch.tensor(X_test[17,:].copy(), dtype=torch.float32).unsqueeze(1), 
    1000
).mean()

## Attack with mean as objective function

In [ ]:
# Plot loss as a function of the size of the perturbation for both deterministic and reparametrization trick attacks
losses_rep_all = []
losses_fgsm_all = []
epsilons = np.linspace(0, .5, 10)
y_star = 9.0

for _ in range(4):
    losses_rep = []
    losses_fgsm = []

    for epsilon in epsilons:
        x_adv = torch.tensor(X_test[17,:].copy(), dtype=torch.float32)
        x_adv_values, loss_values, func_values = attack(x_adv, model, y_star, epsilon=epsilon, learning_rate=1e-4, func=id)  
        y_adv = model.sample_predictive_distribution(torch.tensor(x_adv_values[-1], dtype=torch.float32).unsqueeze(1), 1000).mean()
        losses_rep.append((y_adv - y_star) ** 2)

        x_adv_fgsm = attack_fgsm(torch.tensor(X_test[17,:].copy(), dtype=torch.float32), model, y_star, epsilon=epsilon)
        y_adv_fgsm = model.sample_predictive_distribution(x_adv_fgsm.unsqueeze(1), 1000).mean()
        losses_fgsm.append((y_adv_fgsm - y_star) ** 2)

    losses_rep_all.append(losses_rep)
    losses_fgsm_all.append(losses_fgsm)

# plot the mean and std of the losses
plt.errorbar(epsilons, np.mean(losses_rep_all, axis=0), yerr=2*np.std(losses_rep_all, axis=0), label='Reparametrization trick')
plt.errorbar(epsilons, np.mean(losses_fgsm_all, axis=0), yerr=2*np.std(losses_fgsm_all, axis=0), label='FGSM')
plt.xlabel('$\epsilon$')
plt.ylabel('Loss')
plt.legend()

## Attack with exp(y^2/100) as objective function

In [ ]:
ys = model.sample_predictive_distribution(torch.tensor(X_test[17,:], dtype=torch.float32).unsqueeze(1), 1000)
func = torch.exp(ys ** 2 / 100)
func.mean()

In [ ]:
# Plot loss as a function of the size of the perturbation for reparametrization trick attack
x_adv = torch.tensor(X_test[17,:].copy(), dtype=torch.float32, requires_grad=True)
y_star = 3.0
losses_total_rep = []
losses_total_fgsm = []
epsilons = np.linspace(0, .8, 10)

def compute_loss_rep(epsilon):
    x_adv_values, loss_values, func_values = attack(x_adv, model, y_star, epsilon=epsilon, learning_rate=1e-4, num_iterations=1000, samples_per_iteration=1000, func=expy2)    
    y_adv = model.sample_predictive_distribution(torch.tensor(x_adv_values[-1]).unsqueeze(1), 1000)
    func_mean = torch.exp(y_adv ** 2 / 100).mean()
    return (func_mean - y_star) ** 2

def compute_loss_fgsm(epsilon):
    x_adv_fgsm = attack_fgsm(x_adv, model, y_star, epsilon=epsilon, func=expy2)
    y_adv_fgsm = model.sample_predictive_distribution(x_adv_fgsm.unsqueeze(1), 1000)
    func_mean = torch.exp(y_adv_fgsm ** 2 / 100).mean()
    return (func_mean - y_star) ** 2

for _ in range(4):
    print(_)
    losses_rep = []
    losses_fgsm = []

    for epsilon in epsilons:
        losses_rep.append(compute_loss_rep(epsilon))
        losses_fgsm.append(compute_loss_fgsm(epsilon))
        
    losses_total_rep.append(losses_rep)
    losses_total_fgsm.append(losses_fgsm)

# plot the mean and std of the losses
plt.errorbar(epsilons, np.mean(losses_total_rep, axis=0), yerr=2*np.std(losses_total_rep, axis=0), label='Reparametrization trick')
plt.errorbar(epsilons, np.mean(losses_total_fgsm, axis=0), yerr=2*np.std(losses_total_fgsm, axis=0), label='FGSM')
plt.legend()
plt.xlabel('$\epsilon$')
plt.ylabel('Loss')

#### Compare attacking one point with l1 or l2 projection

In [ ]:
x = torch.tensor(X_test[17,:], dtype=torch.float32)
y_star = 3.0
x_adv_values, loss_values, func_values = attack(x, model, y_star, epsilon=0.3, learning_rate=1e-4, num_iterations=1000, samples_per_iteration=1000, func=expy2, projection=l1_projection)
x_adv_l1 = x_adv_values[-1]
x_adv_values, loss_values, func_values = attack(x, model, y_star, epsilon=0.3, learning_rate=1e-4, num_iterations=1000, samples_per_iteration=1000, func=expy2)
x_adv_l2 = x_adv_values[-1]
print(x,'\n', x_adv_l1,'\n', x_adv_l2)
print(x-x_adv_l1, '\n', x-x_adv_l2)